# Logging Examples

The following cells shows examples of how to use the various logging modules and functions. It also will display the outputs of each function.

## Install Pre-Requisites and Load RAD Modules

First we need to install the pre-requisites and load the RAD modules to be used in this notebook. The `Init.ps1` script will do this for us.

In [5]:
. .\samples\Init.ps1

Run `Get-Command` to list all the functions available in the RAD modules:

In [6]:
Get-Command -Module Rad-Error-Utils
Get-Command -Module Rad-Text-Utils


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Format-RadErrorMessage                             0.0        Rad-Error-Utils
Function        Get-RadErrorMessage                                0.0        Rad-Error-Utils
Function        Get-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Import-RadErrorsFromYaml                           0.0        Rad-Error-Utils
Function        Invoke-TerminatingCommand                          0.0        Rad-Error-Utils
Function        Set-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Confirm-LoggerIsEnabled                            0.0        Rad-Text-Utils
Function        Show-DebugOutput                                   0.0        Rad-Text-Utils
Function        Show-Error                                         0.0        R

## Enable PoshLog Logger

Run the `Init-Logger.ps1` script to enable the PoshLog logger. This will create a `logs` directory in the current directory and create a `PoshLog` logger that will write to the `~/.utils-logs` directory.

> See an example of a log file in the [utils-10140105.log.example](samples/utils-10140105.log.example) file.

In [7]:
. .\samples\Init-Logger.ps1

## Rad-Error-Utils

These are examples of how to use the `Rad-Error-Utils` module.

### Get-RadErrorMessages

`Get-RadErrorMessages` returns all the error codes and messages that are available:

In [28]:
Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



### Get-RadErrorMessage

`Get-RadErrorMessage` returns a specific error message based on the error code passed in. 

> Some messages allow parameters to be passed in to customize the message. More information on this can be found in the [Format-RadErrorMessage](#format-raderrormessage) section.

In [7]:
Get-RadErrorMessage -ErrorCode 101 -Parameters "param1","param2","param3"

Error 102: Invalid parameter: param1. Expected: param2. Actual: param3.. Error Code: 101


- If a message requires parameters but they are not provided, the function will return an error:

In [8]:
Get-RadErrorMessage -ErrorCode 101 

Exception: Error: The template requires 3 parameters, but only 0 were provided.


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 101 

- If the error code is not found, it will return a generic error message:

In [9]:
Get-RadErrorMessage -ErrorCode 200

An unexpected error occurred while executing the command. Error Code: 200


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 200

### Format-RadErrorMessage

`Format-RadErrorMessage` is used to format error messages in a consistent way to improve readability and maintainability.

It is also used to insert parameters into a message. It takes in a message and a list of parameters and returns the message with the parameters inserted. The parameters are inserted into the message in the order they are passed in in the list. Parameters are denoted in the message by `{}` and are required if they are present. Not all messages need to have parameters.

In [15]:
$ErrorMessages = Get-RadErrorMessages
$ErrorCode = "500"

Format-RadErrorMessage -Template $ErrorMessages[$ErrorCode] -Parameters "param1","param2"

An unexpected error occurred while executing the command: [param1]. Error message: [param2]


Error: Command failed: SubmitCode: $ErrorMessages = Get-RadErrorMessages ...

### Import-RadErrorsFromYaml

`Import-RadErrorsFromYaml` is used to import error codes and messages from a YAML file. It takes in a path to a YAML file and returns a hashtable of the error codes and messages. This allows you to add your own custom error codes and messages. The YAML file must be in the following format:

```yaml
  # common_errors.yaml
  errors:
    1000: "Unexpected error"
    1001: "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    1004: "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    1005: "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    1010: "Unsupported operation - The operation {0} is not supported."
```



In [ ]:
Import-RadErrorsFromYaml -YamlFile ".\samples\errors-example.yaml"

Then after importing the YAML file, you can use the `Get-RadErrorMessages` function to get the error messages:

In [50]:
Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



### Set-RadErrorMessages

`Set-RadErrorMessages` allows you to add new error messages. It takes in a hashtable of error codes and messages and adds them to the existing error messages. If the error code already exists, it will overwrite the existing error message.

For comparison purposes, these are the default error error messages available at the start:

In [76]:
# For comparison purposes, these are the default error error messages available

Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



Now, let's add a few new error messages:

In [77]:
$ErrorMessages = @{
    "1000" = "Unexpected error"
    "1001" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "1004" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "1005" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "1010" = "Unsupported operation - The operation {0} is not supported."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           Unexpected error
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



Finally, let's update an existing error message:

In [78]:
$ErrorMessages = @{
    "1000" = "A RAD error message"
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           A RAD error message
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



> There is an optional parameter `-Overwrite` that will overwrite all the existing error messages with the new error messages.

In [16]:
$ErrorMessages = @{
    "2000" = "An unexpected error occurred while executing the command."
    "2001" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "2004" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "2005" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "2010" = "Unsupported operation - The operation {0} is not supported."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages -Overwrite


Name                           Value
----                           -----
2000                           An unexpected error occurred while executing the command.
2001                           FileNotFoundError - Failure to locate or access a specified file or…
2004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
2005                           TypeError - Mismatched data types or unsupported operations on a da…
2010                           Unsupported operation - The operation {0} is not supported.



Now when you run Get-RadErrorMessages you will see the only new error messages:

In [17]:
Get-RadErrorMessages


Name                           Value
----                           -----
2005                           TypeError - Mismatched data types or unsupported operations on a da…
2000                           An unexpected error occurred while executing the command.
2001                           FileNotFoundError - Failure to locate or access a specified file or…
2010                           Unsupported operation - The operation {0} is not supported.
2004                           PermissionDeniedError - Insufficient permissions to perform the {0}…



### Invoke-TerminatingCommand

`Invoke-TerminatingCommand` is used to execute a command and handle any terminating errors that may occur. It takes in a command and a hashtable of parameters to pass to the command. It will invoke the command and if a terminating error occurs, it will handle the error and return a hashtable with the error code, error message, and error details. If no terminating error occurs, it will return the output of the command.

In [18]:
Invoke-TerminatingCommand -Command {az account show --query id -o tsv 2>&1} -ErrorCode '2000' -ErrorParameters @("Subscription Id")

[04:59:52 INF] Executing Command: [az account show --query id -o tsv 2>&1]
az: The term 'az' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
[04:59:52 FTL] An unexpected error occurred while executing the command.. Error Code: 2000
Exception: An unexpected error occurred while executing the command.. Error Code: 2000


Error: Command failed: SubmitCode: Invoke-TerminatingCommand -Command {az account sho ...

## Rad-Text-Utils

These are examples of how to use the `Rad-Text-Utils` module. 

> If a logger is not enabled, the functions will default to use `Write-Host`, `Write-Debug`, or `Write-Warning` accordingly to display in the console.

### Confirm-LoggerIsEnabled

`Confirm-LoggerIsEnabled` is used to confirm that the logger is enabled. It takes in a logger object and returns a boolean value indicating if the logger is enabled or not.

In [8]:
Confirm-LoggerIsEnabled

True


## Show-Title

`Show-Title` is used to display a title in the console. It takes in a text string and displays it in the console with a `#` in front of it. 

> If you have a logger enabled, the function will use `Write-InfoLog` to display and log the title.

In [9]:
Show-Title "This is a title"

[04:48:05 INF] # THIS IS A TITLE


## Show-Output

`Show-Output` is used to display informational messages to the console. 

> If you have a logger enabled, the function will use `Write-InfoLog` to diplay and log the output.

In [10]:
Show-Output "This is an example output"

[04:48:10 INF] This is an example output


## Show-DebugOutput

`Show-DebugOutput` is used to display debug messages in the console.

> If you have a logger enabled, the function will use `Write-DebugLog` to display and log the debug messages.

In [11]:
Show-DebugOutput "This is an example debug output"

## Show-Warning

`Show-Warning` is used to display a warning in the console. It takes in a text string and displays it in the console with a `WARNING:` in front of it.

> If you have a logger enabled, the function will use `Write-WarningLog` to display and log the warning.

In [12]:
Show-Warning "This is an example warning"

[04:48:17 WRN] This is an example warning


## Show-Error

`Show-Error` is used to display non-terminating errors in the console. It takes in a text string and displays it in the console with a `ERROR:` in front of it.

> If you have a logger enabled, the function will use `Write-ErrorLog` to display and log the error.

In [21]:
Show-Error "This is an example non-terminating error"

Show-Error: This is an example non-terminating error
[05:00:39 ERR] This is an example non-terminating error


Error: Command failed: SubmitCode: Show-Error "This is an example non-terminating err ...

## Show-FatalError

`Show-FatalError` is used to display a fatal error in the console. It takes in a text string and displays it in the console with a `FATAL ERROR:` or `Exception` in front of it.

> If you have a logger enabled, the function will use `Write-FatalLog` to display and log the fatal error.

In [19]:
Show-FatalError "This is an example terminating error"

[05:00:35 FTL] This is an example terminating error
Exception: This is an example terminating error


Error: Command failed: SubmitCode: Show-FatalError "This is an example terminating er ...